In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train =False,
    download = True,
    transform=ToTensor(),
)

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [10]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred,y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 1.161669 [   64/60000]
loss: 1.152849 [ 6464/60000]
loss: 0.974717 [12864/60000]
loss: 1.111511 [19264/60000]
loss: 0.982382 [25664/60000]
loss: 1.018511 [32064/60000]
loss: 1.050153 [38464/60000]
loss: 1.002263 [44864/60000]
loss: 1.027922 [51264/60000]
loss: 0.962441 [57664/60000]
Test Error: 
 Accuracy: 66.2%, Avg loss: 0.975075

Epoch 2
----------------------------
loss: 1.039336 [   64/60000]
loss: 1.050102 [ 6464/60000]
loss: 0.856786 [12864/60000]
loss: 1.014646 [19264/60000]
loss: 0.885977 [25664/60000]
loss: 0.923688 [32064/60000]
loss: 0.969372 [38464/60000]
loss: 0.927304 [44864/60000]
loss: 0.946058 [51264/60000]
loss: 0.893661 [57664/60000]
Test Error: 
 Accuracy: 67.6%, Avg loss: 0.900548

Epoch 3
----------------------------
loss: 0.951128 [   64/60000]
loss: 0.980877 [ 6464/60000]
loss: 0.774477 [12864/60000]
loss: 0.947414 [19264/60000]
loss: 0.822478 [25664/60000]
loss: 0.856608 [32064/60000]
loss: 0.913542 [38464/60000]
loss

In [22]:
#모델 저장
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

#모델 불러오기
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)],classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Saved PyTorch Model State to model.pth
Predicted: "Ankle boot", Actual: "Ankle boot"
